share = True -> Link para acessar via huggingface com meu computador as a server
inbrowser = True -> dá o launch direto no Browser, posso adicionar user e senha ('username','password')

In [ ]:
from openai import OpenAI
from IPython.display import Markdown, display
import gradio as gr
from bs4 import BeautifulSoup
import requests 

In [ ]:
system_message = '''
    Você é um assistente que analisa o conteúdo de uma página web e cria um pequeno
    resumo sobre a empresa ou blog mostrado, responda em markdown sem blocos de
    código.
'''

In [ ]:
def fetch_website_contents(url):
    """
    retorna títulos e conteúdos do website;
    limite de 2 mil caracteres de conteúdo
    """
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "Título não encontrado"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + text)[:2_000]

In [ ]:
def resumo(nome, url, model):
    messages = [{'role':'system', 'content':system_message},
                {'role':'user', 'content':fetch_website_contents(url)}]

    client = OpenAI(api_key='ollama', base_url='http://localhost:11434/v1')

    stream = client.chat.completions.create(
        model=model,
        messages=messages,
        stream=True
    )
    result = ''

    for chunk in stream:
        result += chunk.choices[0].delta.content or ''
        yield result

In [ ]:
nome_input = gr.Textbox(label='Nome')
url_input = gr.Textbox(label='url') 
model_selector = gr.Dropdown(['phi3','llama3.2','llama3.1'], label='Selecionar modelo', value='llama3.1')
resposta_output = gr.Markdown(label='Resposta')

view = gr.Interface(
    fn=resumo,
    title='Gerador de Resumos',
    inputs=[nome_input, url_input, model_selector],
    outputs=[resposta_output],
    examples=[
            ['Hugging Face', 'https://huggingface.co','llama3.1'],
            ['Chip Hueyn','https://huyenchip.com/','phi3']
        ],
    flagging_mode='never'
)
view.launch(share=True, auth=('admin_teste','admin_senha'))
